# ChemEcho Tutorial Notebook

## Initial Setup

In [ ]:
# install chemecho
! pip install git+file:/global/homes/t/tharwood/repos/chemecho

  Cloning file:///global/homes/t/tharwood/repos/chemecho to /tmp/pip-req-build-t3kly6oi
  Running command git clone --filter=blob:none --quiet file:///global/homes/t/tharwood/repos/chemecho /tmp/pip-req-build-t3kly6oi
  Resolved file:///global/homes/t/tharwood/repos/chemecho to commit 44c12041b834529032f1964e8f4b68778c595921
  Preparing metadata (setup.py) ... done
  Created wheel for chemecho: filename=chemecho-0.0.0-py3-none-any.whl size=3586 sha256=7649b146419a93ae6cf350c483c0e5bc5c7aa1296d04db475a2ed961e230626b
  Stored in directory: /tmp/pip-ephem-wheel-cache-0956ih2n/wheels/82/85/82/99fa782b0a22d0a90e7a7021979e18c8e81e0060b4c158aa08
Successfully built chemecho


In [15]:
# imports
from chemecho.utils import load_processed_gnps_data, merge_in_nist
from chemecho.featurization import subformula_featurization, build_feature_matrix

ImportError: cannot import name 'subformula_featurization' from 'chemecho.featurization' (/global/homes/t/tharwood/.conda/envs/form_feat/lib/python3.12/site-packages/chemecho/featurization.py)

In [ ]:
polarity = 'negative' # either 'negative' or 'positive'
workdir = '/pscratch/sd/t/tharwood/chemecho_tutorial' # storage location for feature matrices, MS2 libraries, etc.

# embedding parameters
vector_assignment_method = 'blur' # either 'blur' or 'top'. blur assigns all subformula within tolerance, top takes only the best one
max_ppm_error = 5 # max ppm error of the subformula assignment

## Download & Process MS2 Training Data (Libraries)

In [10]:
gnps_cleaned = load_processed_gnps_data(gnps_cleaned_path=f'{workdir}/gnps_cleaned.tsv', convert_spectra=True, polarity=polarity)

# If you have access to commercial NIST library:
merged_lib = merge_in_nist(gnps_cleaned, nist_cleaned_path=f'{workdir}/nist_cleaned.tsv', convert_spectra=True, polarity=polarity)

# if not
# merged_lib = gnps_cleaned

100%|██████████| 138377/138377 [01:01<00:00, 2234.28it/s] 
/global/homes/t/tharwood/.conda/envs/form_feat/lib/python3.12/site-packages/chemecho/utils.py:141: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  nist_cleaned = pd.read_csv(nist_cleaned_path, sep='\t')


100%|██████████| 21604/21604 [00:01<00:00, 14937.08it/s]


In [14]:
print(f"Total spectra: {merged_lib.shape[0]}")
print(f"Unique InchiKeys: {len(merged_lib.inchikey_smiles.unique())}")

Total spectra: 159981
Unique InchiKeys: 24936


## Build Training Data Feature Matrix

In [ ]:
peak_subformula_vectors, nl_subformula_vectors = subformula_featurization(merged_lib, 
                                                                          vector_assignment=vector_assignment_method, 
                                                                          max_ppm_error=max_ppm_error)

In [ ]:
featurized_spectral_data, feature_vector_index_map = build_feature_matrix(peak_subformula_vectors, nl_subformula_vectors)